In [64]:
import pathlib
import pandas as pd 
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
 
import nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playerdashboardbyclutch, playergamelogs, commonplayerinfo, teamplayeronoffdetails
import pandas as pd

In [65]:
class Player(object):
    def __init__(self, PlayerID, TeamID):
        
        self.PlayerID =  PlayerID 
        self.PlayerName = players.find_player_by_id(player_id=self.PlayerID)['full_name']

        self.StartYear = commonplayerinfo.CommonPlayerInfo(player_id= self.PlayerID).get_data_frames()[0]['FROM_YEAR'][0] 
        if self.StartYear < 2010:
            self.StartYear = 2010
            
        self.LastYear = 2021
        self.TeamID = TeamID

    def compile_active_player_data(self):
        
        player_name = []
        points_scored_raw_data=[]
        game_date_raw_data = []
        win_loss_raw_data = []
        free_throws_raw_data = [] 
        feild_goals_2pts_raw_data = []
        feild_goals_3pts_raw_data = []
        matchup_raw_data = []
        location = []
        player_team_average_raw_data = []
        opponent_team_average_raw_data = []
        print(self.StartYear, self.LastYear)
        for year in range((self.StartYear), (self.LastYear)):
            print("-------------------------------------------------")
            season = str(year) + "-" + str(((year+1)-2000))
            playerLogs = playergamelogs.PlayerGameLogs(player_id_nullable=self.PlayerID, season_nullable=season)
            print(season)
            
            game_date_api_data = playerLogs.get_data_frames()[0]["GAME_DATE"]
            win_loss_api_data = playerLogs.get_data_frames()[0]["WL"]
            free_throws_api_data = playerLogs.get_data_frames()[0]["FTM"]
            feild_goals_2pts_api_data = playerLogs.get_data_frames()[0]["FGM"]
            feild_goals_3pts_api_data = playerLogs.get_data_frames()[0]["FG3M"] 
            matchup_api_data = playerLogs.get_data_frames()[0]["MATCHUP"]


            
            player_team_average_raw_data = #TODO: Need to add the player team average raw data
            
            
            for game in range(0, len(game_date_api_data)):
                    player_name.append(self.PlayerName)
                    game_date_raw_data.append(game_date_api_data[game])
                    win_loss_raw_data.append(win_loss_api_data[game])
                    #free_throws_raw_data.append(free_throws_api_data[game]) 
                    #feild_goals_2pts_raw_data.append(feild_goals_2pts_api_data[game])
                    #feild_goals_3pts_raw_data.append(feild_goals_3pts_api_data[game])
                    points_scored_raw_data.append((free_throws_api_data[game])+(feild_goals_2pts_api_data[game]*2)+(feild_goals_3pts_api_data[game]*3))
                    matchup_raw_data.append(matchup_api_data[game])
                    location.append(0)
                    player_team_average_raw_data.append((teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable=season).get_data_frames()[0][])

        
        player_data = {
        "Player Name": player_name,
        "Points Scored": points_scored_raw_data,
        "Game Date": game_date_raw_data,
        "Win/Loss": win_loss_raw_data,
        #"Free Throws": free_throws_raw_data,
        #"Feild Goals 2pts": feild_goals_2pts_raw_data,
        #"Feild Goals 3pts": feild_goals_3pts_raw_data,
        "Matchup": matchup_raw_data,
        "Location": location
        }

        player_dataFrame = pd.DataFrame(data=player_data)
        print(player_dataFrame)

        return(player_dataFrame)


    def get_active_player_data(self):
        return(self.compile_active_player_data())



In [66]:
teams = {
    "ATL": 1,
    "BKN": 2,
    "BOS": 3,
    "CHA": 4,
    "CHI": 5,
    "CLE": 6,
    "DAL": 7,	
    "DEN": 8,
    "DET": 9,
    "GSW": 10,	
    "HOU": 11,	
    "IND": 12,
    "LAC": 13,	
    "LAL": 14,	
    "MEM": 15, 	
    "MIA": 16,	
    "MIL": 17,	
    "MIN": 18,	
    "NOP": 19,
    "NOH": 19,	
    "NYK": 20,	
    "OKC": 21,	
    "ORL": 22,	
    "PHI": 23,	
    "PHX": 24,	
    "POR": 25,	
    "SAC": 26,	
    "SAS": 27,	
    "TOR": 28,	
    "UTA": 29,	
    "WAS": 30
}

In [67]:
#Data Preparation

test = Player(203081).get_active_player_data()

for i in range(0, len(test)):

    if ((i%10)==0):
        print("Processing Game Number:", i, "/" ,len(test))

    if test["Win/Loss"][i] == "W":
           test["Win/Loss"][i] = 1 
    if test["Win/Loss"][i] == "L":
           test["Win/Loss"][i] = 2
    matchup = test["Matchup"][i]
    if " vs. " in matchup:
        test["Location"][i] = 1
    if "@" in matchup:
        test["Location"][i] = 2
    try:   
        matchup = matchup.replace('POR', "")
    except:
        print("Unexpected Error with removing player team")
    try:
        matchup = matchup.replace(" @ ", "")
        matchup = matchup.replace(" vs. ", "")
    except:
        print("Error")
    
    
    test['Matchup'][i]=teams[matchup]


    
opponent = test.pop('Matchup')

test["ATL"] = (opponent == 1)*1.0
test["BKN"] = (opponent == 2)*1.0
test["BOS"] = (opponent == 3)*1.0
test["CHA"] = (opponent == 4)*1.0
test["CHI"] = (opponent == 5)*1.0
test["CLE"] = (opponent == 6)*1.0
test["DAL"] = (opponent == 7)*1.0
test["DEN"] = (opponent == 8)*1.0
test["DET"] = (opponent == 9)*1.0
test["GSW"] = (opponent == 10)*1.0
test["HOU"] = (opponent == 11)*1.0
test["IND"] = (opponent == 12)*1.0
test["LAC"] = (opponent == 13)*1.0
test["LAL"] = (opponent == 14)*1.0
test["MEM"] = (opponent == 15)*1.0
test["MIA"] = (opponent == 16)*1.0
test["MIL"] = (opponent == 17)*1.0
test["MIN"] = (opponent == 18)*1.0
test["NOP"] = (opponent == 19)*1.0
test["NYK"] = (opponent == 20)*1.0
test["OKC"] = (opponent == 21)*1.0
test["ORL"] = (opponent == 22)*1.0
test["PHI"] = (opponent == 23)*1.0
test["PHX"] = (opponent == 24)*1.0
test["POR"] = (opponent == 25)*1.0
test["SAC"] = (opponent == 26)*1.0
test["SAS"] = (opponent == 27)*1.0
test["TOR"] = (opponent == 28)*1.0
test["UTA"] = (opponent == 29)*1.0
test["WAS"] = (opponent == 30)*1.0		



location = test.pop("Location")

test["Home"] = (location == 1)*1.0
test["Away"] = (location == 2)*1.0

outcome = test.pop("Win/Loss")
test["Win"] = (outcome == 1)*1.0
test["Loss"] = (outcome == 2)*1.0


test    



test = test.drop(columns=["Player Name", "Game Date"])


print(test)




2012 2021
-------------------------------------------------
2012-13
-------------------------------------------------
2013-14
-------------------------------------------------
2014-15
-------------------------------------------------
2015-16
-------------------------------------------------
2016-17
-------------------------------------------------
2017-18
-------------------------------------------------
2018-19
-------------------------------------------------
2019-20
-------------------------------------------------
2020-21
        Player Name  Points Scored            Game Date Win/Loss      Matchup  \
0    Damian Lillard             25  2013-04-17T00:00:00        L  POR vs. GSW   
1    Damian Lillard             16  2013-04-16T00:00:00        L    POR @ LAC   
2    Damian Lillard             38  2013-04-14T00:00:00        L    POR @ DEN   
3    Damian Lillard             12  2013-04-12T00:00:00        L  POR vs. OKC   
4    Damian Lillard             48  2013-04-10T00:00:00        

In [68]:
train_dataset = test.sample(frac=0.8, random_state=0)
test_dataset = test.drop(train_dataset.index)

In [69]:
sns.pairplot(train_dataset[["Points Scored"]])

In [70]:
train_stats = train_dataset.describe()
train_stats.pop("Points Scored")
train_stats = train_stats.transpose()
train_stats

#for this data this is not that informative but the mean gives a good impression of the number of times Damian Lillard had played the shown team

,count,mean,std,min,25%,50%,75%,max
ATL,506.0,0.023715,0.152312,0.0,0.0,0.0,0.0,1.0
BKN,506.0,0.021739,0.145975,0.0,0.0,0.0,0.0,1.0
BOS,506.0,0.027668,0.164182,0.0,0.0,0.0,0.0,1.0
CHA,506.0,0.021739,0.145975,0.0,0.0,0.0,0.0,1.0
CHI,506.0,0.023715,0.152312,0.0,0.0,0.0,0.0,1.0
CLE,506.0,0.023715,0.152312,0.0,0.0,0.0,0.0,1.0
DAL,506.0,0.047431,0.212769,0.0,0.0,0.0,0.0,1.0
DEN,506.0,0.051383,0.220997,0.0,0.0,0.0,0.0,1.0
DET,506.0,0.023715,0.152312,0.0,0.0,0.0,0.0,1.0
GSW,506.0,0.045455,0.208505,0.0,0.0,0.0,0.0,1.0


In [71]:
train_labels = train_dataset.pop("Points Scored")
test_labels = test_dataset.pop("Points Scored")

In [72]:
def sigmoid(x):
    return(x - train_stats['mean'])/train_stats['std']
normed_train_data = sigmoid(train_dataset)
normed_test_data = sigmoid(test_dataset)

In [73]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

In [74]:
model = build_model()

In [75]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                2240      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 6,465
Trainable params: 6,465
Non-trainable params: 0
_________________________________________________________________


In [76]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [77]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')
EPOCHS = 1000

history = model.fit(
    normed_train_data, train_labels,
    epochs=EPOCHS, validation_split= 0.2, verbose=0,
    callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

In [78]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
995,NaN,NaN,NaN,NaN,NaN,NaN,995
996,NaN,NaN,NaN,NaN,NaN,NaN,996
997,NaN,NaN,NaN,NaN,NaN,NaN,997
998,NaN,NaN,NaN,NaN,NaN,NaN,998
999,NaN,NaN,NaN,NaN,NaN,NaN,999
